<a href="https://colab.research.google.com/github/Is7ac/ML/blob/main/FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = os.listdir(data_dir)
        self.images = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(data_dir, class_name)
            for image_name in os.listdir(class_dir):
                self.images.append(os.path.join(class_dir, image_name))
                self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = cv2.imread(self.images[idx])
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# Define image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])



In [5]:
# Define neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# Define dataset and dataloader
dataset = CustomDataset(data_dir='/content/drive/MyDrive/chest_xray/train', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define validation dataset and dataloader
val_dataset = CustomDataset(data_dir='/content/drive/MyDrive/chest_xray/val', transform=transform)  # Use the same transform as training
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the neural network
input_size = 128 * 128 * 3  # Image size after resizing and flattening
num_classes = len(dataset.classes)
model = NeuralNetwork(input_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # Calculate average loss and accuracy for validation set
    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = 100 * val_correct / val_total

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {loss.item():.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.2f}%')

Epoch [1/10], Training Loss: 0.0565, Validation Loss: 0.3362, Validation Accuracy: 93.75%
Epoch [2/10], Training Loss: 0.2930, Validation Loss: 0.1971, Validation Accuracy: 93.75%
Epoch [3/10], Training Loss: 0.1044, Validation Loss: 0.4939, Validation Accuracy: 81.25%
Epoch [4/10], Training Loss: 0.3656, Validation Loss: 0.7194, Validation Accuracy: 68.75%
Epoch [5/10], Training Loss: 0.0549, Validation Loss: 0.3540, Validation Accuracy: 87.50%
Epoch [6/10], Training Loss: 0.2384, Validation Loss: 0.6971, Validation Accuracy: 68.75%
Epoch [7/10], Training Loss: 0.4055, Validation Loss: 0.4031, Validation Accuracy: 87.50%
Epoch [8/10], Training Loss: 0.1029, Validation Loss: 0.3165, Validation Accuracy: 87.50%
Epoch [9/10], Training Loss: 0.3302, Validation Loss: 0.1649, Validation Accuracy: 93.75%
Epoch [10/10], Training Loss: 0.0560, Validation Loss: 0.2327, Validation Accuracy: 93.75%


In [8]:
# Define testing dataset and dataloader
test_dataset = CustomDataset(data_dir='/content/drive/MyDrive/chest_xray/test', transform=transform)  # Use the same transform as training
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Testing loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

Accuracy on the test set: 78.53%
